In [32]:
import os
import numpy as np
from PIL import Image
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import seaborn as sns
import torch
import pandas as pd
import torchvision
from tqdm.notebook import tqdm
from torchvision.io import read_image
from torch.utils.data import Dataset
import torchsummary
from torch.optim import lr_scheduler
from pytorch_lightning.callbacks import Callback
import torch.nn as nn
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf")  # For export
sns.reset_orig()
sns.set()

# Tensorboard extension (for visualization purposes later)
%load_ext tensorboard
# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Seed set to 42


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Device: cpu


## Create a training dataset

In [2]:
shots = [16769, 16773]#, 16534, 16769, 16773, 18057]
shot_df = pd.DataFrame([])
for shot in shots:
    df = pd.read_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')
    shot_df = pd.concat([shot_df, df], axis=0)

df_mode = shot_df['mode'].copy()
df_mode[shot_df['mode']=='L-mode']=0
df_mode[shot_df['mode']=='H-mode']=1
df_mode[shot_df['mode']=='ELM']=2
shot_df['mode'] = df_mode
shot_df = shot_df.reset_index(drop=True) #each shot has its own indexing starting from 0 to 2232

In [3]:

class ImageDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations #pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.loc[idx, 'filename']) #proc ne 'filename'?
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
dataset = ImageDataset(annotations=shot_df[['filename', 'mode']], img_dir='/compass/Shared/Users/bogdanov/vyzkumny_ukol')

In [4]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

## Import pretrained model

In [26]:
model_ft = torchvision.models.resnet18(weights='IMAGENET1K_V1')

num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 1.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = torch.nn.Linear(num_ftrs, 1)



model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Here I simply print the structure of model for input with size (3,500,640)

In [16]:
#torchsummary.summary(model_ft, (3,500,640))

### I will try to freeze all layers except 4-th and fc layer

In [30]:
# for name, para in model_ft.named_parameters():
#     print("="*40)
#     print(f"name: {name}")
#     print(para)

In [27]:
for name, para in model_ft.named_parameters():
    if "layer4" in name or "fc" in name:
        continue
    else:
        para.requires_grad = False

## Training

In [33]:
class GenerateCallback(Callback):
    def __init__(self, input_imgs, every_n_epochs=1):
        super().__init__()
        self.input_imgs = input_imgs  # Images to reconstruct during training
        # Only save those images every N epochs (otherwise tensorboard gets quite large)
        self.every_n_epochs = every_n_epochs

    def on_train_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                reconst_imgs = pl_module(input_imgs)
                pl_module.train()
            # Plot and add to tensorboard
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0, 1)
            grid = torchvision.utils.make_grid(imgs, nrow=2, normalize=True, range=(-1, 1))
            trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

In [35]:
def train(model, train_loader, test_loader, val_loader):
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=500,
        
    )
    trainer.fit(model, train_loader, test_loader)
    trainer.logger._log_graph = True  # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None  # Optional logging argument that we don't need

    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
    result = {"test": test_result, "val": val_result}
    return model, result

In [36]:
trained_model, result = train(model_ft, train_dataloader, test_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `ResNet`